In [6]:
!pip install psycopg2

import psycopg2

username = "sueszli"
password = "password"
host = "localhost"
port = "5432"
database = "postgres"

# test connection
conn = psycopg2.connect(
    user=username,
    password=password,
    host=host,
    port=port,
    database=database
)

In [10]:
!chmod +x ./reset.sh && ./reset.sh >> /dev/null
!psql postgres -c "\dt"

./reset.sh: line 10: return: can only `return' from a function or sourced script
Did not find any relations.
ERROR:  cannot drop the currently open database
ERROR:  current user cannot be dropped
           List of relations
 Schema |    Name     | Type  |  Owner  
--------+-------------+-------+---------
 public | badges      | table | sueszli
 public | comments    | table | sueszli
 public | posthistory | table | sueszli
 public | postlinks   | table | sueszli
 public | posts       | table | sueszli
 public | r           | table | sueszli
 public | s           | table | sueszli
 public | t           | table | sueszli
 public | tags        | table | sueszli
 public | users       | table | sueszli
 public | votes       | table | sueszli
(11 rows)



In [8]:
query = "SELECT a,b,c FROM r NATURAL JOIN s NATURAL JOIN t;"
# !psql postgres -c "\timing" -c "$query"

# minimalist
!psql postgres -c "explain (analyze, costs off) $query"

                                        QUERY PLAN                                        
------------------------------------------------------------------------------------------
 Merge Join (actual time=129.194..310.784 rows=1888027 loops=1)
   Merge Cond: ((t.c = s.c) AND (t.a = r.a))
   ->  Sort (actual time=7.767..8.625 rows=10209 loops=1)
         Sort Key: t.c, t.a
         Sort Method: quicksort  Memory: 1394kB
         ->  Seq Scan on t (actual time=0.011..1.602 rows=20000 loops=1)
   ->  Materialize (actual time=121.421..196.331 rows=2082281 loops=1)
         ->  Sort (actual time=121.412..142.946 rows=390733 loops=1)
               Sort Key: s.c, r.a
               Sort Method: external merge  Disk: 8456kB
               ->  Hash Join (actual time=0.521..27.885 rows=390733 loops=1)
                     Hash Cond: (r.b = s.b)
                     ->  Seq Scan on r (actual time=0.021..1.086 rows=20000 loops=1)
                     ->  Hash (actual time=0.486..0.486 rows=2000